In [13]:
#Import file

from openai import OpenAI
import pandas as pd
import tqdm
import json
import unicodedata
import re
client = OpenAI(api_key="") #API key here


# File path, upload the file
file_path = '/Users/ninachen/Desktop/Privacy_Project/privacy_API_code/test_case_1_new.xlsx'
output_file_path = '/Users/ninachen/Desktop/test_case_1_classification.xlsx'

# Read and select columns
info_privacy_df_content_label = pd.read_excel(file_path)
paragraphs_with_keyword_content_label = info_privacy_df_content_label['paragraphs_with_keyword']
paragraphs_with_keyword_content_page = info_privacy_df_content_label['page_num_with_keyword']
material_ID = info_privacy_df_content_label['ID']

In [2]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    # text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')
    return text

In [3]:
#Varibale Control

#Normalize text
normalize = True

#How many times it will run
times = 3

#Verification times
verifier_times = 1

#ChatGPT model version
model_version = "gpt-3.5-turbo"

#ChatGPT verifier model version
verifier_model_version = "gpt-3.5-turbo"

#Verifier On
verification = True

#Way to decide: consistency or majority
consistency = True


In [4]:
def prompt_instru():
    #single rationale input and output
    definition = """
        Below is privacy-related content, and a content sent from a user to privacy expert.
        Your task is to classify the content into one or more of the following categories:
        definition, risk, protection and other.
        
        "Definition. 
        When considering the definition category, check if the content meets any of the following aspects. Meeting any of them qualifies it for the definition category. No indication is needed for analysis.
        - The content metions the key aspects of privacy and describes what they are.
        - The content mentions perspectives on privacy understanding, and describes what the perspective is. 
        - The content provides an explanation or definition of what privacy means and describes what it is. 
        - The content mentions an evolution of privacy concepts and describes the concept.
        - The content mentions the fundamental principles or theories related to privacy and describes the principles or theories. 
        - The content distinguishes between different types of privacy and explains differences."
        
        "Risk.
        When considering the risk category, check if the content meets any of the following aspects. Meeting any of them qualifies it for the risk category. No indication is needed for analysis.
        - The content mentions the consequences or results of privacy violations, privacy failure or data leakage. 
        - The content mentions that a situation, system or a practice are inadequate or have vulnerabilities related to privacy.
        - The content identifies specific vulnerabilities in systems or processes that could be exploited to breach privacy.
        - The content mentions vulnerabilities that can cause excessive or sensitive data collection.
        - The content describes actions that may lead to privacy violations or harm, and it explicitly details the resulting harm."
        
        "Protection.
        When considering the protection category, check if the content meets any of the following aspects. Meeting any of them qualifies it for the protection category. No indication is needed for analysis.
        - The content mentions an action, effort, or regulation that can better protect privacy, for example, better meeting privacy requirements, protect users' data and information.
        - The content mentions a privacy-related law.
        - The content mentions a method for users' data to be safeguarded or secured. 
        - The content mentions a method that can reduce or mitigate privacy risks, such as encryption, limited acess, anonymity etc.
        - The content mentions an action or requirement mandated by privacy-related law. 
        - The content mentions that something is a prerequisite for privacy protection."
        
        "Other.
        Meeting any of them qualifies it for the other category. No indication is needed for analysis.
        - If it doesn't talk about privacy definition, privacy protection, privacy risk, then it belongs to the other type. 
        - If the content resembles a book preface, summary, introduction, or catalog, classify it as other."
        """

    prompt_example = """
        Here are some examples:
        1.
         {'Content': 'There are three key aspects of privacy:\n■  Freedom from intrusion—being left alone\n■  Control of information about oneself\n■  Freedom from surveillance (from being followed, tracked, watched, and eaves- dropped upon)',
          'Rationale': 'The paragraph mentions composition of privacy, so it is classified as privacy definition.',
          'Category': 'Definition'},
        2.
         {'Content': 'Contrast this view with descriptions of privacy as something that can be intruded upon or invaded, where privacy can be understood in terms of a spatial metaphor, such as a zone, that deserves protection. ',
          'Rationale': 'The paragraph mentions how privacy can be understood and the descriptions of privacy, therefore it is classified as privacy definition.',
          'Category': 'Definition'},
        3.
         {'Content': 'Indirect prompting could be leveraged to exfiltrate users’ data (e.g., credentials, personal information) or leak users’ chat sessions. This can be done in interactive chat sessions by persuading users to disclose their data or indirectly via side channels',
          'Rationale': "The paragraph mentions the harm of indirect prompting on users' data. Exfiltrating and leaking chat sessions are privacy violations, thus classified as privacy risk.",
          'Category': 'Risk'},
        4.
         {'Content': 'In particular, the standard practice of ingesting as much NLP data as possible—including from potentially untrusted public sources—exposes fundamental vulnerabilities ranging from data poisoning to privacy.',
          'Rationale': 'The paragraph mentions a vulnerability in a practice and directly links it to privacy. Therefore, it is classified as a privacy risk.',
          'Category': 'Risk'},
        5.
         {'Content': 'Offer a way for people to opt out from mailing lists, advertising, and other secondary uses. Offer a way for people to opt out from features and services that expose personal information.',
          'Rationale': 'It suggests a way for users to reduce personal information exposure, which is considered as solution to protect privacy.',
          'Category': 'Protection'},
        6.
         {'Content': 'Ensures data minimization. The central server only observes anonymous identifiers of COVID-19 positive users without any proximity information. Health authorities learn no information except that provided when a user reaches out to them after being notified.',
          'Rationale': 'It mentions a method to avoid proximity information and the possibiltiy of information learned by authorities, which protects user privacy. Therefore, it is considered as privacy solution.',
          'Category': 'Protection'},
        7.
         {'Content': 'Record sanctions and fines, new regulations, and stunning court decisions have converged to mandate heightened controls \nand accountability from government regulators, industry and standards groups as well as the public..When combined with the volume of data, information privacy, security, protection of trade secrets, and records compliance become complex and critical, high-risk business issues that only executive management can truly fix.',
          'Rationale': "In the first sentence, this mentions government regulation is necessary but it does not mention if this is for privacy. In the second sentence, it mentions the records compliance can be difficult because of multiple aspects including privacy, but it doesn't mention the purpose of the compliance is for privacy. Therefore, the content does not meet any criteria of the risk, protection, definition categories. It belongs to other.",
          'Category': 'Other'},
        8.
         {'Content': 'Failure to meet them can literally put an organization out of business or land executives in prison..Privacy,\nsecurity, records management, information technology (IT), and business manage-\nment functions are important—very important—but the most signifi cant aspect of all\nof these functions relates to legality and regulatory compliance.',
          'Rationale': "The first sentence mentions a consequence but it is not clear what is the cause of the consequence, therefore it is hard to judge whether this is related to privacy or not. The second sentence mentions privacy and other things are important, especially in an area. But it doesn't talk about definition, risk or protection. Therefore, it belongs to other.",
          'Category': 'Other'}
        Return a single json format with two fields: "Rationale" and "Category" in that order.
            """
    prompt_instru_output = definition + prompt_example
    return prompt_instru_output  

In [5]:
#Single input

def process_single_output(results, input_content, consistency, times):
    predictions = []
    category_list = []
    nums = []
    for result in results:
        try:
            category = eval(result)['Category']
            category_list.append(category)
        except:
            try:
                category = json.loads(result)['Category']
                category_list.append(category)
            except:
                predictions.append("error")
                unsure_list.append("error")
                print("\nInitial Ouput Error")
                return predictions

    if len(category_list) == times:
        print("category_list" + str(category_list))
        for number in range(len(category_list)):
            if isinstance(category_list[number], list):
                category_list[number] = sorted(category_list[number])
            else:
                pass
        
        count = {}
        for item in category_list:
            if isinstance(item, list):
                item_str = str(item)  # Convert list to string for dictionary key
            else:
                item_str = item  # Use the item directly if it's not a list
                
                # Increment the count for the item
            if item_str in count:
                count[item_str] += 1
            else:
                count[item_str] = 1
                
        sorted_dict = sorted(count.items(), key=lambda item:item[1])

        if consistency == True:
            if sorted_dict[-1][1] == times:
                predictions.append(sorted_dict[-1][0])
                return predictions
            else:
                predictions.append("unsure")
                return predictions 
        else:
            if sorted_dict[-1][1] > times/2:
                predictions.append(sorted_dict[-1][0])
                return predictions
            else:
                predictions.append("unsure")
                return predictions 
                          
    else:
        predictions.append("error")
        print("less than 2 output")
        return predictions

In [6]:
def prompt_verifier (prediction_single):
    prompt_final = []
    beginning = "You are a privacy expert responsible for determining if content matches the category definition. Below are the judgment guidelines:"
    definition_vq = "Answer whether the content meets the questions with yes or no. Definition category criteria questions: \n \
    1. Does the content metion the key aspects of privacy and describes what they are?\n \
    2. Does the content mention perspectives on privacy understanding, and describe what the perspective is? \n \
    3. Does the content provide an explanation or definition of what privacy means and describes what it is? \n \
    4. Does the content mention an evolution of privacy concepts and describe the concept? \n \
    5. Does the content mention the fundamental principles or theories related to privacy and describes the principles or theories?\n \
    6. Does the content distinguishes between different types of privacy and explains differences?"
        
    risk_vq = "Answer whether the content meets the questions with yes or no. Risk category criteria questions: \n \
    1. Does the content mention the consequences or results of privacy violations, privacy failure or data leakage? \n \
    2. Does the content mention that a situation, system or a practice are inadequate or have vulnerabilities related to privacy?\n \
    3. Does the content identify specific vulnerabilities in systems or processes that could be exploited to breach privacy? \n \
    4. Does the content mention vulnerabilities that can cause excessive or sensitive data collection? \n \
    5. Does the content describe actions that may lead to privacy violations or harm, and it explicitly details the resulting harm?"

    protection_vq = "Answer whether the content meets the questions with yes or no. Protection category criteria questions: \n \
    1. Does the content mention an action, effort, or regulation that can better protect privacy, for example, better meeting privacy requirements, protect users' data and information?\n \
    2. Does the content mention a privacy-related law?\n \
    3. Does the content mention a method for users' data to be safeguarded or secured?\n \
    4. Does the content mention a method that can reduce or mitigate privacy risks, such as encryption, limited acess, anonymity etc?\n \
    5. Does the content mention an action or requirement mandated by privacy-related law?\n \
    6. Does the content mention that something is a prerequisite for privacy protection?"

    other_vq = "Answer whether the content meets the questions with yes or no. Other category critera questions:\n \
    1. Does the content metion the key aspects of privacy and describes what they are?\n \
    2. Does the content mention perspectives on privacy understanding, and describe what the perspective is? \n \
    3. Does the content provide an explanation or definition of what privacy means and describes what it is? \n \
    4. Does the content mention an evolution of privacy concepts and describe the concept? \n \
    5. Does the content mention the fundamental principles or theories related to privacy and describes the principles or theories?\n \
    6. Does the content distinguishes between different types of privacy and explains differences?\n \
    7. Does the content mention the consequences or results of privacy violations, privacy failure or data leakage? \n \
    8. Does the content mention that a situation, system or a practice are inadequate or have vulnerabilities related to privacy?\n \
    9. Does the content identify specific vulnerabilities in systems or processes that could be exploited to breach privacy? \n \
    10. Does the content mention vulnerabilities that can cause excessive or sensitive data collection? \n \
    11. Does the content describe actions that may lead to privacy violations or harm, and it explicitly details the resulting harm?\n \
    12. Does the content mention an action, effort, or regulation that can better protect privacy, for example, better meeting privacy requirements, protect users' data and information?\n \
    13. Does the content mention a privacy-related law?\n \
    14. Does the content mention a method for users' data to be safeguarded or secured?\n \
    15. Does the content mention a method that can reduce or mitigate privacy risks, such as encryption, limited acess, anonymity etc?\n \
    16. Does the content mention an action or requirement mandated by privacy-related law?\n \
    17. Does the content mention that something is a prerequisite for privacy protection?"
    
    ending = "Return a single json dictionary format with one fields: 'List'. Put 'yes' or 'no' for each criteria question into the 'List'. Example format: {'List': ['yes', 'yes', 'no']}."

    prompt_list = []
    for category in prediction_single:
        normalized_string = category.lower()
        if normalized_string == "definition":
            prompt = beginning + definition_vq + ending
            prompt_list.append(prompt)
        
        elif normalized_string == "risk":
            prompt = beginning + risk_vq + ending
            prompt_list.append(prompt)

        elif normalized_string == "protection":
            prompt = beginning + protection_vq + ending
            prompt_list.append(prompt)
        
        elif normalized_string == "other":
            prompt = beginning + other_vq + ending
            prompt_list.append(prompt)
        else:
            prompt_list.append("error")
    return prompt_list


In [7]:
#Verifier final judgements

def verifier_final_result(index, verification_result):
    final_judgement = []
    normalized_answer = prediction_single[index]
    parse_fail = 0
    
    try:
        verification_dict = eval(verification_result[0])
    except:
        judgement = False
        final_judgement.append(judgement)
        
    print("verification_dict"+str(verification_dict))
    if normalized_answer.lower() == "other":
        if any(answer == "yes" for answer in verification_dict['List']):
            judgement = False
            final_judgement.append(judgement)
        else:
            judgement = True
            final_judgement.append(judgement)
            
    else:
        if any(answer == "yes" for answer in verification_dict['List']):
            judgement = True
            final_judgement.append(judgement)
        else:
            judgement = False
            final_judgement.append(judgement)
    return final_judgement

In [8]:
#Verifier

def verification_process(prediction_single, verifier_model_version, input_content):
    print("--------Start Verification--------")
    output_verify_predication = []
    prediction_second = []
    for index, verification_q in enumerate(prompt_verifier(prediction_single)):
        verification_result = []
        if verification_q != "error":
            completion = client.chat.completions.create(
                    model= verifier_model_version,
                    messages=[
                  # System Prompt, see https://platform.openai.com/docs/guides/text-generation/chat-completions-api
                  {"role": "system", "content": verification_q},
                  {"role": "user", "content": "Does the category match the content (yes or no only)? {'Content': " + input_content +"}, {'Category':"+str(prediction_single[index])}
                    ],
                    temperature = 0.7
                )
            verification_result.append(completion.choices[0].message.content)
            print("verification_result"+str(verification_result))

            final_decisions = verifier_final_result(index, verification_result)
            print("final decisions" +str(final_decisions))
            if any(decision == False for decision in final_decisions):
                prediction_second.append("unsure")
                print("Fail verification")
            else:
                prediction_second.append(prediction_single[index])
        else:
            prediction_second.append("error")
    for item in prediction_second:
        if item == "error":
            output_verify_predication = ["error"]
        elif item == "unsure":
            output_verify_predication = ["unsure"]
        else:
            output_verify_predication = prediction_second 
    return output_verify_predication

In [9]:
# Run the classification

final_predictions_with_verification = []
final_unsure_list = []
final_predictions_without_verification = []

for i in tqdm.trange(len(paragraphs_with_keyword_content_label)):
    results= []
    if normalize == True:
        input_content = normalize_text(paragraphs_with_keyword_content_label[i])
    else:
        input_content = paragraphs_with_keyword_content_label[i]
    for _ in range(times):
        completion = client.chat.completions.create(
                model= model_version,
                messages=[
              # System Prompt, see https://platform.openai.com/docs/guides/text-generation/chat-completions-api
              {"role": "system", "content": prompt_instru()},
              {"role": "user", "content": "What's the catagory? {'Content': " + input_content +"}"}
                ],
                temperature = 0.7
            )

        # Adding generated answers to results
        results.append(completion.choices[0].message.content)

    prediction_single = process_single_output(results, i, consistency, times)

    #List without verification
    final_predictions_without_verification.append(prediction_single)
    
    if verification == True:
        if any(pred == "error" or pred == "unsure" for pred in prediction_single):
            final_predictions_with_verification.append(prediction_single)   
        else:
            prediction_verification = verification_process(prediction_single, verifier_model_version, input_content)
            final_predictions_with_verification.append(prediction_verification)
        
    else:
        final_predictions_without_verification.append(prediction_single)

    print("Final predictions without verification" + str(final_predictions_without_verification[-1]))
    print("Final predictions with verification" + str(final_predictions_with_verification[-1]))

  0%|                                                    | 0/40 [00:00<?, ?it/s]

category_list['Protection', 'Protection', 'Protection']
--------Start Verification--------


  2%|█                                           | 1/40 [00:04<02:55,  4.51s/it]

verification_result["{'List': ['yes', 'no', 'yes', 'yes', 'no', 'no']}"]
type<class 'list'>
verification_dict{'List': ['yes', 'no', 'yes', 'yes', 'no', 'no']}
final decisions[True]
Final predictions without verification['Protection']
Final predictions with verification['Protection']
category_list['Protection', 'Protection', 'Protection']
--------Start Verification--------


  5%|██▏                                         | 2/40 [00:08<02:44,  4.33s/it]

...

verification_result["{'List': ['yes', 'no', 'no', 'no', 'yes']}"]
type<class 'list'>
verification_dict{'List': ['yes', 'no', 'no', 'no', 'yes']}
final decisions[True]
Final predictions without verification['Risk']
Final predictions with verification['Risk']
category_list['Risk', 'Risk', 'Risk']
--------Start Verification--------


 98%|█████████████████████████████████████████▉ | 39/40 [02:49<00:03,  3.47s/it]

verification_result["{'List': ['yes', 'yes', 'yes', 'no', 'yes']}"]
type<class 'list'>
verification_dict{'List': ['yes', 'yes', 'yes', 'no', 'yes']}
final decisions[True]
Final predictions without verification['Risk']
Final predictions with verification['Risk']
category_list['Risk', 'Risk', 'Risk']
--------Start Verification--------


100%|███████████████████████████████████████████| 40/40 [02:53<00:00,  4.35s/it]

verification_result["{'List': ['no', 'no', 'no', 'no', 'no']}"]
type<class 'list'>
verification_dict{'List': ['no', 'no', 'no', 'no', 'no']}
final decisions[False]
Fail verification
Final predictions without verification['Risk']
Final predictions with verification['unsure']


In [17]:
sure_list = []
unsure_list = []
prediction_input_control = final_predictions_with_verification

for n in range(len(prediction_input_control)):
    for m in prediction_input_control [n]:
        if m == "unsure" or m == "error":
            result_unsure = {"ID": material_ID[n], "page_num_with_keyword": paragraphs_with_keyword_content_page[n], "paragraphs_with_keyword": paragraphs_with_keyword_content_label[n], "classification": m}
            unsure_list.append(result_unsure)
            continue
        else:
            if not isinstance(prediction_input_control[n], list):
                prediction_category = list([prediction_input_control[n].lower().replace('privacy','').strip()])
            else:
                prediction_category = sorted([i.lower().replace('privacy','').strip() for i in prediction_input_control[n]])
            result_sure = {"ID": material_ID[n], "page_num_with_keyword": paragraphs_with_keyword_content_page[n], "paragraphs_with_keyword": paragraphs_with_keyword_content_label[n], "classification": prediction_category}
            sure_list.append(result_sure)
            
# Convert sure_list and unsure_list to DataFrames
sure_df = pd.DataFrame(sure_list)
unsure_df = pd.DataFrame(unsure_list)

# Save results to a new Excel file
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    sure_df.to_excel(writer, sheet_name='Sure Classification', index=False)
    
    # Save unsure_list to a new sheet
    if not unsure_df.empty:
        unsure_df.to_excel(writer, sheet_name='Unsure Classification', index=False)
        
print("Already Output Classification File")

Already Output Classification File
